# Baseline Model

## Table of Contents
1. [Load dataset](#load-dataset)
2. [Feature Selection](#feature-selection)
3. [Implementation](#implementation)
4. [Evaluation](#evaluation)


In [ ]:
!pip install openpyxl

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import shutil
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error


## 1. Load dataset
Load preprocessed dataset and print basic details.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the dataset
# Replace 'your_dataset.csv' with the path to your actual dataset
df = pd.read_csv('/content/drive/MyDrive/opencampus_all_files/combined_data_r1.csv')

## 2. Feature Selection

Selected features describes terrain characteristics and cumulative statistics of current track.

In [ ]:
# Feature selection
# Example: Selecting only two features for a simple baseline model
X = df[['Elevation', 'Slope_prev', 'Slope_next', 'Angle', 'Distance', 'Cumulative_Slope']]
y = df['Speed']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Size of datasets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


##  3. Implementation
In following notebook two models were implemented:
- Linear Regression
- Neural Network





#### Baseline model 1 - Linear Regression

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Initialize and train the model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Make predictions
y_pred = lr_model.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

In [ ]:
# Export ml model
import joblib

# Assuming 'lr_model' is your trained Linear Regression model
# Save the model to a file
filename = 'linear_regression_model.joblib'
joblib.dump(lr_model, filename)

#### Baseline model 2 - Simple Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the neural network model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(1)  # Output layer (no activation since we're doing regression)
])

# Compile the model
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
loss, mae = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Optionally, plot the training history (e.g., loss or MAE over epochs)
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

## 4. Evaluation

Evaluation based on loss and MAE.


#### Preprocess test file

In [ ]:
# unzip test files
!unzip /content/drive/MyDrive/opencampus_all_files/Rider1_test.zip -d /content

In [ ]:
# Feature selection
df = pd.read_csv('/content/content/Rider1_test/f15.csv')

# Example: Selecting only two features for a simple baseline model
real_time = df['Time']
X = df[['Elevation', 'Slope_prev', 'Slope_next',  'Angle', 'Distance', 'Cumulative_Slope']] # Cumulative slope
y = df['Speed']


#### Evaluation 1 - Linear Regression

In [ ]:
# Make predictions on the new data
y_pred = lr_model.predict(X)

# Add the predicted values as a new column 'Speed_pred' in the original DataFrame
df['Speed_pred'] = y_pred

# Optionally, check the first few rows to verify the new column is added correctly
print(df.head())


In [ ]:
# Calculate MAE between df['Speed_pred'] and df['Speed']
mae = mean_absolute_error(df['Speed'], df['Speed_pred'])
print(f"Mean Absolute Error: {mae}")


#### Evaluation 2 - Neural Network

In [ ]:
# Normalize the new data using the same scaler fitted on the training data
X_new_scaled = scaler.transform(X)

# Make predictions on the new data
y_pred = model.predict(X_new_scaled)

# Add the predicted values as a new column 'Speed_pred' in the original DataFrame
df['Speed_pred'] = y_pred

# Optionally, check the first few rows to verify the new column is added correctly
print(df.head())


In [ ]:
# prompt: Calculate MAE between df['Speed_pred'] and df['Speed']

mae = mean_absolute_error(df['Speed'], df['Speed_pred'])
print(f"Mean Absolute Error: {mae}")


In [ ]:
df.head()

### Evaluate track length in seconds

In [ ]:
# Initialize Time column
df['pred_time'] = 0.0

# Compute predicted time
for i in range(2, len(df)):
  if pd.notna(df.loc[i, 'Speed_pred']):
    if df.loc[i, 'Speed_pred'] != 0:
      df.loc[i, 'pred_time'] = df.loc[i - 1, 'pred_time'] + (df.loc[i, 'Distance'] - df.loc[i - 1, 'Distance']) / df.loc[i, 'Speed_pred']
    else:
      df.loc[i, 'pred_time'] = df.loc[i - 1, 'pred_time']

print(df)

In [ ]:
df[['Speed', 'Speed_pred', 'Time', 'pred_time']]